 ## Load and Visualize Point Cloud

In [16]:
import laspy
import open3d as o3d
import numpy as np

# Load .las file
las_file = laspy.read("./data/ukazka.las")  # Replace with your file path
# Extracting the x, y, and z coordinates from the .las file
points = np.vstack((las_file.x, las_file.y, las_file.z)).transpose()

# Create Open3D point cloud from the extracted points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Check if color information is available in the .las file
if hasattr(las_file, 'red') and hasattr(las_file, 'green') and hasattr(las_file, 'blue'):
    # Extracting and normalizing color information
    colors = np.vstack((las_file.red, las_file.green, las_file.blue)).transpose() / 65535.0
    pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the entire point cloud
o3d.visualization.draw_geometries([pcd]) 


 ## Create and Visualize a Slice at a Defined Plane

In [17]:
# Define the slicing plane (e.g., z = 0.5)
bottom_plane_z = 410
top_plane_z = 411

# Convert the point cloud to a numpy array
pcd_np = np.asarray(pcd.points)

# Filtering the points based on the z-coordinate to be between the two planes
filtered_indices = np.where((pcd_np[:, 2] > bottom_plane_z) & (pcd_np[:, 2] < top_plane_z))[0]

# Creating a new point cloud from the filtered indices
sliced_pcd = pcd.select_by_index(filtered_indices)

# Visualize the sliced section of the point cloud
o3d.visualization.draw_geometries([sliced_pcd])


 ## Separate and Visualize Green Points

In [18]:
def is_green(point):
    # Check if the green value is greater than both red and blue values
    r, g, b = point[0], point[1], point[2]
    return g > r and g > b

# Check again for color information availability
if hasattr(las_file, 'red') and hasattr(las_file, 'green') and hasattr(las_file, 'blue'):
    # Converting color information to numpy array
    colors_np = np.asarray(pcd.colors)

    # Identifying indices of green points
    green_indices = [i for i, color in enumerate(colors_np) if is_green(color)]

    # Creating a new point cloud for green points
    green_pcd = o3d.geometry.PointCloud()
    green_pcd.points = o3d.utility.Vector3dVector(pcd_np[green_indices])
    green_pcd.colors = o3d.utility.Vector3dVector(np.array([[0, 1, 0] for _ in green_indices]))  # Assigning green color

    # Identifying indices of non-green points
    non_green_indices = [i for i, color in enumerate(colors_np) if not is_green(color)]

    # Creating a new point cloud for non-green points
    non_green_pcd = o3d.geometry.PointCloud()
    non_green_pcd.points = o3d.utility.Vector3dVector(pcd_np[non_green_indices])
    non_green_pcd.colors = o3d.utility.Vector3dVector(np.array([[1, 0, 0] for _ in non_green_indices]))  # Assigning another color (e.g., red)

    # Visualize the point cloud with green and non-green points
    o3d.visualization.draw_geometries([green_pcd, non_green_pcd])
else:
    print("Color information not available in the .las file.")


RuntimeError: 

 ## Separate and Visualize based on image

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter

# Load the image
image = cv2.imread('./data/colors.png')
# Convert from BGR to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Reshape the image to be a list of pixels
pixels = image.reshape((-1, 3))

# We use KMeans to cluster the pixel colors
kmeans = KMeans(n_clusters=1)  # We want to find one cluster as you specified
kmeans.fit(pixels)

# Get the RGB values of the cluster center
dominant_color = kmeans.cluster_centers_.astype(int)[0]
print(f"The dominant color in the image is: {dominant_color}")


The dominant color in the image is: [88 94 94]


d:\Owncloud\Local_Home\vyuka\PV_231211\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
# Define a color similarity threshold
color_threshold = 10  # This is an example threshold value

# Function to determine if a point's color is similar to the dominant color
def is_similar_color(point_color, target_color, threshold):
    return all(abs(pc - tc) < threshold for pc, tc in zip(point_color, target_color))

# Extract color information of the point cloud if available
if hasattr(las_file, 'red') and hasattr(las_file, 'green') and hasattr(las_file, 'blue'):
    # Normalize the color information for comparison
    point_cloud_colors = np.vstack((las_file.red, las_file.green, las_file.blue)).transpose() / 65535.0
    point_cloud_colors = (point_cloud_colors * 255).astype(int)  # Scale color values to 0-255

    # Find the indices of points with colors similar to the dominant color
    similar_color_indices = [i for i, color in enumerate(point_cloud_colors) if is_similar_color(color, dominant_color, color_threshold)]

    # Create a new point cloud for points with similar colors
    similar_color_pcd = o3d.geometry.PointCloud()
    similar_color_pcd.points = o3d.utility.Vector3dVector(np.array(pcd.points)[similar_color_indices])
    similar_color_pcd.colors = o3d.utility.Vector3dVector(np.array([[c/255.0 for c in dominant_color] for _ in similar_color_indices]))

    # Visualize the point cloud with filtered colors
    o3d.visualization.draw_geometries([similar_color_pcd])
else:
    print("Color information not available in the .las file.")
